In [1]:
## Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")

In [3]:
from llama_index import VectorStoreIndex,SimpleDirectoryReader
documents=SimpleDirectoryReader("data").load_data()

In [1]:
!pip install FAISS



ERROR: Could not find a version that satisfies the requirement FAISS (from versions: none)
ERROR: No matching distribution found for FAISS

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\lenovo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
index=VectorStoreIndex.from_documents(documents,show_progress=True)

e:\New Recordings\Langchain Videos\LLAmindex\Projects\Llama_index\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 36/36 [00:03<00:00, 11.11it/s]


In [6]:
index

In [7]:
query_engine=index.as_query_engine()

In [22]:
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

retriever=VectorIndexRetriever(index=index,similarity_top_k=4)
postprocessor=SimilarityPostprocessor(similarity_cutoff=0.80)

query_engine=RetrieverQueryEngine(retriever=retriever,
                                  node_postprocessors=[postprocessor])


In [23]:
response=query_engine.query("What is attention is all yopu need?")

In [24]:
from llama_index.response.pprint_utils import pprint_response
pprint_response(response,show_source=True)
print(response)

Final Response: The paper "Attention Is All You Need" proposes a new
network architecture called the Transformer. This architecture is
based solely on attention mechanisms and does not use recurrent or
convolutional neural networks. The paper demonstrates that the
Transformer models outperform existing models in terms of quality,
parallelizability, and training time. The Transformer achieves state-
of-the-art results in machine translation tasks and generalizes well
to other tasks such as English constituency parsing.
______________________________________________________________________
Source Node 1/1
Node ID: be144ab8-cb0a-44fa-af69-3dbfe555e41a
Similarity: 0.8107415810551661
Text: Provided proper attribution is provided, Google hereby grants
permission to reproduce the tables and figures in this paper solely
for use in journalistic or scholarly works. Attention Is All You Need
Ashish Vaswani∗ Google Brain avaswani@google.comNoam Shazeer∗ Google
Brain noam@google.comNiki Parmar∗ Goo

In [25]:
import os.path
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = faiss.IndexFlatL2(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

# either way we can now query the index
query_engine = index.as_query_engine()
response = query_engine.query("What are transformers?")
print(response)

Transformers are a model architecture that rely entirely on an attention mechanism to draw global dependencies between input and output. They eschew recurrence and do not use sequence-aligned RNNs or convolution. Transformers allow for significantly more parallelization and have been shown to achieve state-of-the-art results in tasks such as translation. They can be trained faster than architectures based on recurrent or convolutional layers.


In [2]:
## Retrieval augmented generation

import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")

# Install FAISS and SentenceTransformer
!pip install faiss-cpu
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Step 1: Load your documents
# Assuming SimpleDirectoryReader("data").load_data() returns a list of document texts
from llama_index import SimpleDirectoryReader
documents = SimpleDirectoryReader("data").load_data()
document_texts = [doc['text'] for doc in documents]  # Adjust based on actual structure

# Step 2: Convert documents to embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')  # You can choose an appropriate model
document_embeddings = model.encode(document_texts)

# Step 3: Build the FAISS index
dimension = document_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)  # Using L2 distance
index.add(np.array(document_embeddings))

# Step 4: Querying the FAISS index
class FAISSQueryEngine:
    def __init__(self, index, model, documents):
        self.index = index
        self.model = model
        self.documents = documents
    
    def query(self, query_text, top_k=4, similarity_cutoff=0.80):
        query_embedding = self.model.encode([query_text])
        distances, indices = self.index.search(np.array(query_embedding), k=top_k)
        
        results = []
        for i, idx in enumerate(indices[0]):
            if distances[0][i] <= similarity_cutoff:
                results.append((self.documents[idx], distances[0][i]))
        return results

query_engine = FAISSQueryEngine(index, model, document_texts)

# Example query
response = query_engine.query("What is attention is all you need?")
for res, dist in response:
    print(f"Document with distance {dist}: {res}")

# Persisting the FAISS index (if needed)
faiss.write_index(index, 'faiss_index.bin')

# Loading the FAISS index (if needed)
# index = faiss.read_index('faiss_index.bin')
# query_engine = FAISSQueryEngine(index, model, document_texts)


TypeError: str expected, not NoneType